# SINIFLANDIRMA

Problem;

Çiçek türlerinde yaprak genişliği ve yaprak uzunluğu bakımından farklar vardır. Biz bu yaprak genişliği ve yaprak uzunlukları ile çiçeğin hangi çiçek türünden olduğunu en iyi nasıl tahmin edecebiliriz ona bakacağız.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
.master("local[4]")  \
.appName("İris") \
.config("spark.driver.memory", "2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

In [4]:
df = spark.read.format("csv") \
.option("header", True) \
.option("sep",",") \
.option("inferSchema", True) \
.load("C:\\Users\\busra\\Desktop\\Egitim\\Apache Spark\\iriss.csv")

In [5]:
df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


5 farklı değişken vardır. İlk dört değişken yaprakların genişliği ve uzunluğu ile bilgiler verir. Son değişkenimiz ise çiçek türünü bize söylemektedir.

In [6]:
df.describe().toPandas().head()

,summary,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,count,150,150,150,150,150
1,mean,5.843333333333335,3.0540000000000007,3.7586666666666693,1.1986666666666672,None
2,stddev,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,None
3,min,4.3,2.0,1.0,0.1,Iris-setosa
4,max,7.9,4.4,6.9,2.5,Iris-virginica


Çıktıya baktığımzda count değerleri tüm değişkenlerde aynı çıkmıştır yani veride boşluk yoktur.

In [7]:
import pyspark.sql.functions as f

In [8]:
df.groupBy("Species").agg(f.count("*").alias("sayi")).show()

+---------------+----+
|        Species|sayi|
+---------------+----+
|    Iris-setosa|  50|
|Iris-versicolor|  50|
| Iris-virginica|  50|
+---------------+----+



3 çiçek türü vardır ve hepsinden 50 tane bulunmaktadır. Sınıflar dengeli dağılmıştır.

In [9]:
df.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



-Kategorik nitelikli değişken olmadığı için  OneHotEncoder yapmıyoruz.

-Boş değer olmadığı için imputer yapmıyoruz.

-Sadece hedef değişken(Species) Kategorik olduğu için string indexer kullanacağız.

In [10]:
df.columns

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer().setHandleInvalid("skip") \
.setInputCol("Species") \
.setOutputCol("label")

In [13]:
indexerDF = indexer.fit(df).transform(df)

In [14]:
indexerDF.toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0


Burada String indexer ile label değişkenini oluşturduk. Label değişkeni Species kategorik değişkeninin numeric halidir. 

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
degiskenler = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

In [17]:
assembler = VectorAssembler().setInputCols(degiskenler).setOutputCol("features")

In [18]:
assembler_df = assembler.transform(indexerDF)

In [19]:
assembler_df.toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0,"[5.1, 3.5, 1.4, 0.2]"
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0,"[4.9, 3.0, 1.4, 0.2]"
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.3, 0.2]"
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0,"[4.6, 3.1, 1.5, 0.2]"
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0,"[5.0, 3.6, 1.4, 0.2]"


Burada ise Vector assembler ile features değişkenini oluşturduk. Bu değişken ilk dört değişkenin birleştirilmiş halidir. 

In [20]:
train_df, test_df =assembler_df.randomSplit([0.8,0.2],seed=142)

In [21]:
train_df.count()

123

In [22]:
test_df.count()

27

Şimdi model oluşturma aşamasına geçilmiştir

In [23]:
from pyspark.ml.classification import LogisticRegression

In [24]:
logreg_obj = LogisticRegression() \
.setLabelCol("label") \
.setFeaturesCol("features")

In [25]:
logreg_model = logreg_obj.fit(train_df)

In [26]:
transformed_df = logreg_model.transform(test_df)

In [27]:
transformed_df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features,rawPrediction,probability,prediction
0,4.4,3.2,1.3,0.2,Iris-setosa,0.0,"[4.4, 3.2, 1.3, 0.2]","[1156.7057793868014, -175.61828154657348, -981...","[1.0, 0.0, 0.0]",0.0
1,4.6,3.6,1.0,0.2,Iris-setosa,0.0,"[4.6, 3.6, 1.0, 0.2]","[1431.2111137582035, -252.65946587203337, -117...","[1.0, 0.0, 0.0]",0.0
2,4.7,3.2,1.6,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.6, 0.2]","[1066.5956068607993, -138.74814303662134, -927...","[1.0, 0.0, 0.0]",0.0
3,4.8,3.4,1.6,0.2,Iris-setosa,0.0,"[4.8, 3.4, 1.6, 0.2]","[1179.3497704455172, -179.7268068157752, -999....","[1.0, 0.0, 0.0]",0.0
4,5.0,3.0,1.6,0.2,Iris-setosa,0.0,"[5.0, 3.0, 1.6, 0.2]","[899.0238895107, -42.054436933733825, -856.969...","[1.0, 0.0, 0.0]",0.0


Çıktıya bakıldığında Prediction olarak yeni bir değişken oluşmuştur. Bu değişekn label değişkeninin tahminidir.

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
evaluator = MulticlassClassificationEvaluator() \
.setLabelCol("label") \
.setPredictionCol("prediction") \
.setMetricName("accuracy")

In [30]:
accuracy = evaluator.evaluate(transformed_df)

In [31]:
accuracy

0.8888888888888888

Lojistik regresyon %88 doğrulukla tahminlerimizi yapmıştır.

In [32]:
from pyspark.ml.classification import NaiveBayes

In [33]:
nb_obj = NaiveBayes() \
.setLabelCol("label") \
.setFeaturesCol("features")

In [34]:
nb_model = nb_obj.fit(train_df)

In [35]:
transformed_nb_df = nb_model.transform(test_df)

In [36]:
transformed_nb_df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features,rawPrediction,probability,prediction
0,4.4,3.2,1.3,0.2,Iris-setosa,0.0,"[4.4, 3.2, 1.3, 0.2]","[-10.939880007598047, -12.203818904267164, -12...","[0.6990417853863115, 0.19750652516100842, 0.10...",0.0
1,4.6,3.6,1.0,0.2,Iris-setosa,0.0,"[4.6, 3.6, 1.0, 0.2]","[-10.93487628408366, -12.668718971057965, -13....","[0.7927565705200739, 0.1400043529564869, 0.067...",0.0
2,4.7,3.2,1.6,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.6, 0.2]","[-11.733059048913503, -12.83202612574861, -13....","[0.6631208028890265, 0.22096185906832788, 0.11...",0.0
3,4.8,3.4,1.6,0.2,Iris-setosa,0.0,"[4.8, 3.4, 1.6, 0.2]","[-12.021304102893517, -13.24687466259062, -13....","[0.6929908297026491, 0.20345527597556393, 0.10...",0.0
4,5.0,3.0,1.6,0.2,Iris-setosa,0.0,"[5.0, 3.0, 1.6, 0.2]","[-11.727060941388215, -12.768391208357569, -13...","[0.650338750416613, 0.22955969730546621, 0.120...",0.0


In [37]:
evaluator = MulticlassClassificationEvaluator() \
.setLabelCol("label") \
.setPredictionCol("prediction") \
.setMetricName("accuracy")

In [38]:
accuracy_nb = evaluator.evaluate(transformed_nb_df)

In [39]:
accuracy_nb

0.8518518518518519

Naive Bayes %85 doğrulukla tahminlerimizi yapmıştır.

İki sınıflandırma modeli karşılaştırıldığında Lojistik regresyon modelinin tercih edilmesi daha doğru olmaktadır.

# KÜMELEME

Problem;

Müşterilerin harcama skoru verilmiştir. Kısaca açıklamak gerekirse en az harcayan müşteriye 1 skoru, en çok harcayan müşteriye ise 99 skoru verilmiştir. Burada müşteriler harcama skorlarına göre gruplandırılmak isteniyor. Bunun sebebi az alışveriş yapan müşteriye çok yapması için bir uygulama yapılmak istenmesidir.

In [40]:
import findspark
findspark.init()

In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

In [42]:
spark = SparkSession.builder\
.appName("malcustomer") \
.master("local[4]")\
.getOrCreate()

In [43]:
df = spark.read.format("csv") \
.option("header",True) \
.option("sep",";") \
.option("inferSchema",True) \
.load("C:\\Users\\busra\\Desktop\\Egitim\\Apache Spark\\customer.csv")

In [44]:
df.limit(5).toPandas().head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [45]:
df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)



In [46]:
df.describe().show()

+-------+------------------+------+-----------------+------------------+----------------------+
|summary|        CustomerID|Gender|              Age|Annual Income (k$)|Spending Score (1-100)|
+-------+------------------+------+-----------------+------------------+----------------------+
|  count|               200|   200|              200|               200|                   200|
|   mean|             100.5|  null|            38.85|             60.56|                  50.2|
| stddev|57.879184513951124|  null|13.96900733155888| 26.26472116527124|    25.823521668370173|
|    min|                 1|Female|               18|                15|                     1|
|    max|               200|  Male|               70|               137|                    99|
+-------+------------------+------+-----------------+------------------+----------------------+



Id, gender, age değişkenlerinin etkisinin olmadığını düşünüyoruz o yüzden onları modelimize almayacağız.

In [47]:
vector_assembler = VectorAssembler() \
.setInputCols(["Annual Income (k$)" , "Spending Score (1-100)"]) \
.setOutputCol("features")

Burada iki değişken birleştirip features olarak yeni bir değişken oluşturulmuştur.

In [48]:
standard_scaler = StandardScaler() \
.setInputCol("features") \
.setOutputCol("scaled_features")

Oluşturduğumuz yeni değişkeni standartlaştırdık.

In [57]:
def runKMeans(df,k):
    kmeans_obj = KMeans() \
    .setK(k) \
    .setSeed(142) \
    .setFeaturesCol("scaled_features") \
    .setPredictionCol("cluster")
    
    pipeline_obj = Pipeline() \
    .setStages([vector_assembler,standard_scaler,kmeans_obj])
    
    pipeline_model = pipeline_obj.fit(df)
    return pipeline_model

In [58]:
for k in range(2,11):
    pipeline_model = runKMeans(df, k)
    transformed_df = pipeline_model.transform(df) 
    
    evaluator = ClusteringEvaluator() \
    .setFeaturesCol("scaled_features") \
    .setPredictionCol("cluster") \
    .setMetricName("silhouette")
    
    score=evaluator.evaluate(transformed_df)
    
    print(k,score)

2 0.4527066677349661
3 0.6288672765684974
4 0.657293825903269
5 0.7408351139612729
6 0.7263562509896634
7 0.7236275795916086
8 0.688490882807354
9 0.6258921742873865
10 0.6372440905182243


k= 5 olduğunda en yüksek değeri almıştır. Artık k'yı 5 seçebiliriz.

In [51]:
kmeans_obj = KMeans() \
.setK(5) \
.setSeed(142) \
.setFeaturesCol("scaled_features") \
.setPredictionCol("cluster")

In [52]:
pipeline_obj = Pipeline() \
.setStages([vector_assembler,standard_scaler,kmeans_obj])

In [53]:
pipeline_model = pipeline_obj.fit(df)

In [54]:
transformed_df = pipeline_model.transform(df)

In [55]:
transformed_df.limit(5).toPandas().head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100),features,scaled_features,cluster
0,1,Male,19,15,39,"[15.0, 39.0]","[0.5711082903036444, 1.510251022337088]",4
1,2,Male,21,15,81,"[15.0, 81.0]","[0.5711082903036444, 3.1366752002385674]",2
2,3,Female,20,16,6,"[16.0, 6.0]","[0.6091821763238874, 0.2323463111287828]",4
3,4,Female,23,16,77,"[16.0, 77.0]","[0.6091821763238874, 2.9817776594860455]",2
4,5,Female,31,17,40,"[17.0, 40.0]","[0.6472560623441304, 1.5489754075252185]",4


Features değişkenini vector assembler ile , scaled_features değişkenini standardizasyon ile, cluster değişkenini ise prediction ile oluşturduk. 

In [56]:
transformed_df.groupBy("cluster").count().show()

+-------+-----+
|cluster|count|
+-------+-----+
|      4|   23|
|      2|   22|
|      1|   81|
|      3|   39|
|      0|   35|
+-------+-----+



Görüldüğü gibi değişkenimiz sınıflara ayrılmıştır. Her bir sınıfa belirli bir gözlem atanmıştır.